In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
import math as mt
from scipy.sparse.linalg import * #used for matrix multiplication
from scipy.sparse.linalg import svds
from scipy.sparse import csc_matrix
from sklearn.metrics import mean_squared_error
from math import sqrt
import time
def compute_svd(urm, K):
    U, s, Vt = svds(urm, K)  # 这里的s是一个个值，所以下面要转成矩阵的形式

    dim = (len(s), len(s))
    S = np.zeros(dim, dtype=np.float32)
    for i in range(0, len(s)):
        S[i, i] = mt.sqrt(s[i])

    U = csc_matrix(U, dtype=np.float32)
    S = csc_matrix(S, dtype=np.float32)
    Vt = csc_matrix(Vt, dtype=np.float32)

    return U, S, Vt

def user_item(test):  # 得到键值对，用户：借过的书籍
    ui = dict(test)
    groups = test.groupby(['user_id'])
    for item, group in groups:
        ui[item] = set(group.loc[:, 'book_id'])
    return ui

def Recommend(test_data,user,topk,svd_p_df):
    items = list(set(list(test_data['book_id'])))
    predictList = [svd_p_df.loc[user,item] for item in items]
    series = pd.Series(predictList, index=items)
    series = series.sort_values(ascending=False)[:topk]
    return series

def recallAndPrecision(test,ui,svd_p_df):  # 召回率和准确率
    userID = set(test.loc[:, 'user_id'])
    hit = 0
    recall = 0
    precision = 0
    for userid in userID:
            # trueItem = test[test.ix[:,0] == userid]
            # trueItem= trueItem.ix[:,1]
        trueItem = ui[userid]
        preitem = Recommend(test_data,userid,5,svd_p_df)
        for item in list(preitem.index):
            if item in trueItem:
                hit += 1
        recall += len(trueItem)
        precision += len(preitem)
    return (hit / (recall * 1.0), hit / (precision * 1.0))

start=time.clock()
train_data = pd.read_csv('positive_negtive_data\positive_negtive_data_19_2VS3.csv')
test_data= pd.read_csv('user_book_score_time\\user_book_score_time_19_3VS2.csv',usecols=['user_id','book_id','final_score'])
ui = user_item(test_data)
n_users = train_data.user_id.nunique()                   # 用户、物品数去重统计
n_items = train_data.book_id.nunique()
print('用户个数为{}，图书数目为{}'.format(n_users,n_items))

users = test_data.user_id.nunique()                   # 用户、物品数去重统计
items = test_data.book_id.nunique()
print('用户个数为{}，图书数目为{}'.format(users,items))

train_data_array = train_data.final_score.values
train_row_array = train_data.user_id.values
train_col_array = train_data.book_id.values

test_data_array =test_data.final_score.values
test_row_array = test_data.user_id.values
test_col_array = test_data.book_id.values

train_data_sparse = coo_matrix((train_data_array, (train_row_array, train_col_array)),dtype=float)
test_data_sparse = coo_matrix((test_data_array, (test_row_array, test_col_array)),dtype=float)

n_users_list = train_data.user_id.unique()
print('用户列表为：{}'.format(n_users_list))

K=40
urm = train_data_sparse
MAX_PID = urm.shape[1]
MAX_UID = urm.shape[0]

U, S, Vt = compute_svd(urm, K)
svd_prediction = np.dot(np.dot(U, S), Vt)
svd_p_df = pd.DataFrame(svd_prediction.todense())

recall,precision=recallAndPrecision(test_data,ui,svd_p_df)
print('召回率为：{}'.format(recall))
print('准确率为:{}'.format(precision))
prediction_flatten = svd_prediction[train_data_sparse.todense().nonzero()]
train_data_matrix_flatten = train_data_sparse.todense()[train_data_sparse.todense().nonzero()]
sqrt_train_val=sqrt(mean_squared_error(prediction_flatten, train_data_matrix_flatten))
print('训练数据均方误差为:{}'.format(sqrt_train_val))

test_prediction_flatten = svd_prediction[test_data_sparse.todense().nonzero()]
test_data_matrix_flatten = test_data_sparse.todense()[test_data_sparse.todense().nonzero()]
sqrt_test_val=sqrt(mean_squared_error(test_prediction_flatten, test_data_matrix_flatten))
print('测试数据均方误差为:{}'.format(sqrt_test_val))
duration=time.clock()-start
print('耗费时间：{}'.format(duration))

用户个数为244，图书数目为381
用户个数为244，图书数目为381
用户列表为：[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 